In [1]:
from gurobipy import *
import time
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import shapefile as shp
from collections import defaultdict
from datetime import datetime

In [2]:
maxOpenD = 8
maxOpenA = 2
maxDist_D = 150 #km (1 degree is about 70km)
maxDist_A = 350 #km (1 degree is about 70km)

#maxPopD- max capacity of each district court
#maxPopA - max capacity of each appeals court

#langS
#langD
#langA

### Extract Data

In [3]:
#Get Settlement List
Sfile = "prov_code_24_25_26"
#Sfile = "afg_ppl_settlement_pnt" 

Settlements=pd.read_csv("%s.csv"%Sfile,sep=",")
Settlements = Settlements[["OBJECTID","POPULATION", "LAT_Y","LON_X"]]
#Settlements=Settlements[[0,10,11]]
S = Settlements.shape[0]
Settlement_List = Settlements['OBJECTID'].tolist()

#Get District Court List
Districts = pd.read_csv("District_Courts.csv",sep=",")
Districts = Districts[["DIST_CODE","POP","LON_X", "LAT_Y"]]
#Districts = Districts[[1,4,5]]
D = Districts.shape[0]
District_List = Districts['DIST_CODE'].tolist()

#Get Appeals Court List
Appeals = pd.read_csv("Appeals_Courts.csv",sep=",")
Appeals = Appeals[["PROV_CODE","POP","LON_X", "LAT_Y"]]
#Appeals = Appeals[[1,4,5]]
A = Appeals.shape[0]
Appeals_List = Appeals['PROV_CODE'].tolist()

#### Create Data Subset for Settlements

#### ------------------------------------------------

In [4]:
#Settlements = Settlements.sample(frac = 0.02, replace = False, random_state = 23)
# = Settlements.shape[0]
#Settlement_List = Settlements['OBJECTID'].tolist()

### Set Parameters

####  ------------------------------------------------

In [5]:
#Create Dictionaries
Settlement_Dict = Settlements.set_index('OBJECTID').T.to_dict('list')
District_Dict = Districts.set_index('DIST_CODE').T.to_dict('list')
Appeals_Dict = Appeals.set_index('PROV_CODE').T.to_dict('list')

In [6]:
#Create Dictionaries for District Courthouse Distances

Dist_D = {}

R = 6371e3


for d in District_List:
    d_lon = District_Dict[d][1]
    d_lat = District_Dict[d][2]
    phi2 = np.radians(d_lat)
    

    for s in Settlement_List:
        s_lon = Settlement_Dict[s][2]
        s_lat = Settlement_Dict[s][1]
        phi1 = np.radians(s_lat)
        
        #Distance to District Court
        delta_phi = np.radians(d_lat - s_lat)
        delta_lambda = np.radians(d_lon - s_lon)
        a = np.sin(delta_phi/2) * np.sin(delta_phi/2) + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda/2) * np.sin(delta_lambda/2)
        c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
        
        Dist_D[s,d] = (R * c)/1000
    

In [7]:
#Create Dictionaries for Appeals Courthouse Distances

Dist_A = {}

R = 6371e3

for a in Appeals_List:
    a_lon = Appeals_Dict[a][1]
    a_lat = Appeals_Dict[a][2]
    theta_3 = np.radians(a_lat)

    for s in Settlement_List:
        s_lon = Settlement_Dict[s][2]
        s_lat = Settlement_Dict[s][1]
        theta_1 = np.radians(s_lat)
        
        #Distance to District Court
        delta_theta_ = np.radians(a_lat - s_lat)
        delta_lambda = np.radians(a_lon - s_lon)
        a1 = np.sin(delta_theta_/2) * np.sin(delta_theta_/2) + np.cos(theta_1) * np.cos(theta_3) * np.sin(delta_lambda/2) * np.sin(delta_lambda/2)
        c = 2 * np.arctan2(np.sqrt(a1), np.sqrt(1-a1))
        
        Dist_A[s,a] = (R * c)/1000
        
    

### Create Model

In [8]:
#Create Model
LP = Model("Afg_LP")

LP.Params.OutputFlag = 1
LP.Params.LogFile="log_LPIP_%s.log"%Sfile

Parameter OutputFlag unchanged
   Value: 1  Min: 0  Max: 1  Default: 1
Changed value of parameter LogFile to log_LPIP_prov_code_24_25_26.log
   Prev: gurobi.log  Default: 


### Create Variables

##### Helper function

In [9]:
# Return value of variable
def VarVal(var):
    if (type(var) == gurobipy.Var):
        val = var.X
    else:
        val = 0
    return val

##### d_ij

In [10]:
# Create d_i_j variables
d_LP = {}
for i in Settlement_List:
    d_LP[i] = {}
    for j in District_List:
        if Dist_D[i,j] < maxDist_D:
            d_LP[i][j] = LP.addVar(vtype=GRB.CONTINUOUS, lb=0, ub=1, name='d_LP_%s_%s' % (i, j))
            #d[i][j].start=10000
        else:
            d_LP[i][j] = 0
LP.update()

##### a_ik

In [11]:
 # Create a_i_k variables
a_LP = {}
for i in Settlement_List:
    a_LP[i] = {}
    for k in Appeals_List:
        a_LP[i][k] = {}
        if Dist_A[i,k] < maxDist_A:
            a_LP[i][k] = LP.addVar(vtype=GRB.CONTINUOUS,lb=0,ub=1, name='a_LP_%s_%s' % (i, k))
            #a[i][k].start=0
        else:
            a_LP[i][k] = 0
LP.update()

##### c_jk

In [12]:
# Create c_j_k variables
c_LP = {}
for j in District_List:
    c_LP[j] = {}
    for k in Appeals_List:
        c_LP[j][k] = LP.addVar(vtype=GRB.CONTINUOUS, lb=0, ub=1, name='c_LP_%s_%s' % (j, k))
LP.update()

##### openD

In [13]:
#Create openD variables
openD_LP = {}
for j in District_List:
    openD_LP[j] = LP.addVar(vtype=GRB.CONTINUOUS, lb=0, ub=1, name='openD_LP_%s' % (j))
LP.update()

##### openA

In [14]:
#Create openA variables
openA_LP = {}
for k in Appeals_List:
    openA_LP[k] = LP.addVar(vtype=GRB.CONTINUOUS, lb=0, ub=1, name='openA_LP_%s' % (k))
LP.update()

### Create Constraints

##### D_ij row sums and column sums

In [15]:
#One S -> D Assignment
for i in Settlement_List:
    LP.addConstr(quicksum(d_LP[i][j] for j in District_List) == 1)
LP.update()

In [16]:
#Maximum District Courts Open
for j in District_List:
    LP.addConstr(quicksum(d_LP[i][j] for i in Settlement_List) <= S * openD_LP[j])
LP.update()

##### A_ik row sums and column sums

In [17]:
#One S -> A Assignment
for i in Settlement_List:
    LP.addConstr(quicksum(a_LP[i][k] for k in Appeals_List) == 1)
LP.update()

In [18]:
#Maximum Appeals Courts Open IP
for k in Appeals_List:
    LP.addConstr(quicksum(a_LP[i][k] for i in Settlement_List) <= S * openA_LP[k])
LP.update()

#### C_jk row sums and column sums

In [19]:
#One D -> A Assignment LP
for j in District_List:
    LP.addConstr(quicksum(c_LP[j][k] for k in Appeals_List) == openD_LP[j])
LP.update()

In [20]:
#C constraints
for k in Appeals_List:
    LP.addConstr(quicksum(c_LP[j][k] for j in District_List) <= D * openA_LP[k])
LP.update()

##### Max Open Courthouse Constraints

In [ ]:
#LP Constraints
LP.addConstr(quicksum(openD_LP[j] for j in District_List) <= maxOpenD)
LP.addConstr(quicksum(openA_LP[k] for k in Appeals_List) <= maxOpenA)
LP.update()

##### Linking Constraints

In [ ]:
LP.addConstrs(d_LP[i][j] + a_LP[i][k] - c_LP[j][k] <= 1 for i in Settlement_List for j in District_List for k in Appeals_List)
LP.update()

LP.addConstrs(d_LP[i][j] + c_LP[j][k] - a_LP[i][k] <= 1 for i in Settlement_List for j in District_List for k in Appeals_List)
LP.update()

In [ ]:
# for k in Appeals_List:
#     print("\nAppeal Court %s\n"%k)
#     for j in District_List:
#         for i in Settlement_List:
#             LP.addConstr(d_LP[i][j] + a_LP[i][k] - c_LP[j][k] <= 1)
#             LP.addConstr(d_LP[i][j] + c_LP[j][k] - a_LP[i][k] <= 1)
#     LP.update()

In [ ]:
LP

### Set Objective Function

#### D_ij and A_ik

In [ ]:
LP.setObjective(
        quicksum(   quicksum(   Dist_D[i,j]*d_LP[i][j] for i in Settlement_List)    for j in District_List) + \
        quicksum(   quicksum(   Dist_A[i,k]*a_LP[i][k] for i in Settlement_List)    for k in Appeals_List), GRB.MINIMIZE)
LP.update()

### Optimize

In [ ]:
#Initiate while loop
start_time = time.time()

LP.optimize()

end_time = time.time()

print(end_time-start_time)
   

In [ ]:
print("red\nyellow")

### Output Solution

In [ ]:
print(end_time-start_time)
LP.write("out_LPIP_%s.sol"%Sfile)

###  Plotting

#### DOUT

In [ ]:
# DOUT - District Assignments

DOUT = pd.DataFrame.from_dict({(i,j): VarVal(d_IP[i][j]) 
                           for  i in d_IP.keys() 
                           for j in d_IP[i].keys()},
                           orient='index')

Settlements_DOUT = [i[0] for i in DOUT.index]
Districts_DOUT = [i[1] for i in DOUT.index]

DOUT['Settlement'] = Settlements_DOUT
DOUT['DistrictCourts'] = Districts_DOUT

DOUT = DOUT[DOUT[0] == 1.0]

#### AOUT

In [ ]:
#AOUT - Appeals Assignments

AOUT = pd.DataFrame.from_dict({(i,j): VarVal(a_IP[i][j]) 
                           for i in a_IP.keys() 
                           for j in a_IP[i].keys()},
                           orient='index')

Settlements_AOUT = [i[0] for i in AOUT.index]
Appeals_AOUT = [i[1] for i in AOUT.index]

AOUT['Settlement'] = Settlements_AOUT
AOUT['AppealsCourts'] = Appeals_AOUT

AOUT = AOUT[AOUT[0] == 1.0]

#### COUT

In [ ]:
#COUT - Appeals Assignments

COUT = pd.DataFrame.from_dict({(i,j): VarVal(c_IP[i][j]) 
                           for i in c_IP.keys() 
                           for j in c_IP[i].keys()},
                           orient='index')

Districts_COUT = [i[0] for i in COUT.index]
Appeals_COUT = [i[1] for i in COUT.index]

COUT['DistrictCourts'] = Districts_COUT
COUT['AppealsCourts'] = Appeals_COUT

COUT = COUT[COUT[0] == 1.0]

#### Plot all Locations

In [ ]:
plt.scatter(Settlements['LON_X'],Settlements['LAT_Y'])
plt.scatter(Districts['LON_X'],Districts['LAT_Y'], color='Red', marker = 's')
plt.scatter(Appeals['LON_X'],Appeals['LAT_Y'], color='Green', marker = '^')
plt.show()

#### Plotting Assignments

In [ ]:
#Drawing Lines
#plt.figure(figsize=(14,14))

#Plotting Points    
plt.scatter(Settlements['LON_X'],Settlements['LAT_Y'])
plt.scatter(Districts['LON_X'],Districts['LAT_Y'], color='Red', marker = 's',s=25)
plt.scatter(Appeals['LON_X'],Appeals['LAT_Y'], color='Green', marker = '^',s = 400)

for index,row in DOUT.iterrows():
    s = row['Settlement'] 
    d = row['DistrictCourts']
    #Get District number that Settlement is linked to
    Dist = Districts.loc[Districts['DIST_CODE'] == d]
    Sett = Settlements.loc[Settlements['OBJECTID'] == s]
    X = [Sett.iloc[0,3],Dist.iloc[0,2]]
    Y = [Sett.iloc[0,2],Dist.iloc[0,3]]
    plt.scatter(Dist.iloc[0,2],Dist.iloc[0,3], color='Orange', marker = 's',s = 100)
    plt.plot(X,Y,zorder=1, color="Black")

axes = plt.gca()
axes.set_xlim([69,71])
axes.set_ylim([34,36])

plt.show()

In [ ]:
#Draw Lines
plt.figure(figsize=(14,14))

for index,row in COUT.iterrows():
    d = row['DistrictCourts'] 
    a = row['AppealsCourts']

    Dist = Districts.loc[Districts['DIST_CODE'] == d]
    App = Appeals.loc[Appeals['PROV_CODE'] == a]

    X = [Dist.iloc[0,2],App.iloc[0,2]]
    Y = [Dist.iloc[0,3],App.iloc[0,3]]

    plt.plot(X,Y,zorder=1, color="Black")
    
clr = cm.rainbow(np.linspace(0, 1, D))

for a in range(A):
    X = Appeals.iloc[a,2]
    Y = Appeals.iloc[a,3]
    plt.scatter(X,Y, color="Green", marker = '^',s = 50)
    
for index,row in DOUT.iterrows():   
    d = row['DistrictCourts'] 
    Dist = Districts.loc[Districts['DIST_CODE'] == d]
    X = Dist.iloc[0,2]
    Y = Dist.iloc[0,3]
    c = Dist.index[0]
    plt.scatter(X,Y, color=clr[c], marker = 's')
    
for index,row in DOUT.iterrows():
    s = row['Settlement'] 
    d = row['DistrictCourts']
    #Get District number that Settlement is linked to
    Dist = Districts.loc[Districts['DIST_CODE'] == d]
    Sett = Settlements.loc[Settlements['OBJECTID'] == s]

    c = Dist.index[0]
    X = Sett.iloc[0,3]                                                     
    Y = Sett.iloc[0,2]
    plt.scatter(X,Y, color=clr[c])
          
axes = plt.gca()
axes.set_xlim([66,71])
axes.set_ylim([34,38])

sf = shp.Reader("Afghanistan_Districts","rb")
for shape in sf.shapeRecords():
    x = [i[0] for i in shape.shape.points[:]]
    y = [i[1] for i in shape.shape.points[:]]
    plt.plot(x,y,color='k',linewidth=0.1)
plt.show()